In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import math
import time
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../src')
from preprocessing import *
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import SkewedChi2Sampler
from sklearn.kernel_approximation import PolynomialCountSketch 
# LOL esta está en sklearn pero no me deja importarla
# https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.PolynomialCountSketch.html#

ImportError: cannot import name 'PolynomialCountSketch' from 'sklearn.kernel_approximation' (/home/man3/.local/lib/python3.8/site-packages/sklearn/kernel_approximation.py)

In [24]:
df_db = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', '../datasets/raw/HT_Sensor_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [16]:
df_train, df_test = split_series_byID(100, 0.75, df_db)

In [17]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain, ytrain = df_train[features].values, df_train['class'].values
xtest, ytest = df_test[features].values, df_test['class'].values
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(702938, 11)
(702938,)
(226053, 11)
(226053,)


## Radial Basis Function

In [22]:
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(xtrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_features, ytrain)

X_features = rbf_feature.fit_transform(xtest)
score = clf.score(X_features, ytest)
print(score)

0.813269454508456


## Additive Chi 2

In [21]:
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
X_transformed = chi2sampler.fit_transform(xtrain, ytrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_transformed, ytrain)

X_transformed = chi2sampler.fit_transform(xtest, ytest)
score = clf.score(X_transformed, ytest)
print(score)

0.8368966569786731


## Skewed Chi 2

In [32]:
chi2_feature = SkewedChi2Sampler(skewedness=.01, n_components=10, random_state=0)
X_features = chi2_feature.fit_transform(xtrain, ytrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_features, ytrain)

X_features = chi2_feature.fit_transform(xtest, ytest)
score = clf.score(X_features, ytest)
print(score)

0.8507739335465576


## Polinomial Count 

In [33]:
ps = PolynomialCountSketch(degree=3, random_state=1)
X_features = ps.fit_transform(xtrain)
clf = SGDClassifier(max_iter=100, tol=1e-4)
clf.fit(X_features, ytrain)

X_features = ps.fit_transform(xtest)
score = clf.score(X_features, ytest)
print(score)

NameError: name 'PolynomialCountSketch' is not defined